In [1]:
import numpy as np
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from scipy.stats import linregress

from sklearn.model_selection import KFold, LeaveOneOut

import time

np.random.seed(12345)

# Set number of folds

In [2]:
# Comment out if using Leave One Out
K = 12

In [3]:
import numpy as np
import pandas as pd

from scipy.integrate import odeint, solve_ivp
from scipy.optimize import minimize

import jax
import jax.numpy as jnp
from jax import jit, jacfwd, vmap, random
from jax.experimental.ode import odeint

# import MCMC library
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, HMC

from tqdm import tqdm

### Basic usage ###

'''
# import ODE
from autode.autode import ODE

# instantiate ODE fit
model = ODE(system, df, params)

# fit to data
params = model.fit()

where df has columns [Time, Treatments, S1, ..., SN]
'''

# define function that returns model sensitivity vector
def runODE(t_eval, x, params, ctrl_params, dX_dt):
    # solve ODE model
    y = odeint(dX_dt, x, t_eval, params, ctrl_params)
    return y

# function to compute ODE gradients
def dZdt(system, Z, t, x, params, ctrl_params):

    # compute Jacobian (gradient of model w.r.t. x)
    Jx = jacfwd(system, 1)(t, x, params, ctrl_params)

    # compute gradient of model w.r.t. parameters
    Jp = jacfwd(system, 2)(t, x, params, ctrl_params)

    return Jx@Z + Jp

# define function that returns model sensitivity vector
def runODEZ(t_eval, x, params, ctrl_params, dXZ_dt):
    # check dimensions
    dim_x = len(x)
    n_params = len(params)
    dim_z = dim_x*n_params

    # set initial condition to z equal to zeros
    xz = jnp.concatenate((x, np.zeros(dim_z)))

    # solve ODE model
    y = odeint(dXZ_dt, xz, t_eval, params, ctrl_params)

    return y

### Function to process dataframes ###

def process_df(df, sys_vars, measured_vars, controls):
    # store treatment names
    all_treatments = df.Treatments.values
    unique_treatments = np.unique(all_treatments)

    # store measured datasets for quick access
    data = []
    for i,treatment in enumerate(unique_treatments):

        # pull community trajectory
        comm_inds = np.in1d(df['Treatments'].values, treatment)
        comm_data = df.iloc[comm_inds].copy()

        # make sure comm_data is sorted in chronological order
        comm_data.sort_values(by='Time', ascending=True, inplace=True)

        # pull evaluation times
        t_eval = np.array(comm_data['Time'].values, float)

        # pull initial condition
        Y_init = np.array(comm_data[sys_vars].values[0], float)
        
        # pull system data
        Y_measured = np.array(comm_data[measured_vars].values, float)

        # pull control params
        ctrl_params = np.array(comm_data[controls].values, float)

        # append t_eval and Y_measured to data list
        data.append([t_eval, Y_init, Y_measured, ctrl_params])

    return data

class ODE:
    def __init__(self, system, dataframes, compressors, params, sys_vars, measured_vars, controls = [],
                 alpha_0=1., prior=None, verbose=True):
        '''
        system: a system of differential equations

        dfs: dataframes each with columns
        [Treatment], [Time], [x_1], ..., [x_n], [control_1], ..., [control_m]

        sys_vars: List of variable names of all model outputs as they appear in
                  dataframe (df). (Includes measured and unobserved outputs)

        params: initial guess of model parameters

        measured_sys_vars: List of observed (measured) model outputs

        control_param

        '''

        # make sure params are 1-dimensional
        self.params = np.array(params).ravel()
        if prior is not None:
            self.prior = np.array(prior).ravel()
        else:
            self.prior = np.zeros_like(params)

        # initial degree of regularization
        self.alpha_0 = alpha_0

        # number of parameters
        self.n_params = len(params)

        # dimension of model output
        self.sys_vars = sys_vars
        self.measured_vars = measured_vars
        self.n_sys_vars = len(sys_vars)

        # control input
        self.controls = controls
        self.n_ctrls = len(controls)

        # set compressors 
        self.compressors = []
        for compressor in compressors:
            # vmap over all time points 
            self.compressors.append(jit(vmap(compressor)))
        
        # store derivative of compressors 
        self.compressor_primes = []
        for compressor in compressors:
            # vmap over all time points 
            self.compressor_primes.append(jit(vmap(jacfwd(compressor))))
            
        # set up data
        self.datasets = []
        for i,(df, measured_var) in enumerate(zip(dataframes, measured_vars)):
            self.datasets.append(process_df(df, sys_vars, measured_var, controls))
            
        # for additional output messages
        self.verbose = verbose

        # set parameters of precision hyper-priors
        self.a = 1e-4
        self.b = 1e-4

        # set posterior parameter precision and covariance to None
        self.A = None
        self.Ainv = None

        # jit compile differential equation
        def dX_dt(x, t, params, ctrl_params):
            # concatentate x and z
            return system(t, x, params, ctrl_params)
        self.dX_dt = jit(dX_dt)

        # if not vectorized, xz will be 1-D
        dim_z = len(sys_vars)*len(params)
        def dXZ_dt(xz, t, params, ctrl_params):
            # split up x and z
            x = xz[:len(sys_vars)]
            Z = jnp.reshape(xz[len(sys_vars):], [len(sys_vars), len(params)])
            dzdt = jnp.reshape(dZdt(system, Z, t, x, params, ctrl_params), dim_z)

            # concatentate x and z
            dXZdt = jnp.concatenate([system(t, x, params, ctrl_params), dzdt])
            return dXZdt
        self.dXZ_dt = jit(dXZ_dt)

        # jit compile function to integrate ODE
        self.runODE  = jit(lambda t_eval, x, params, ctrl_params: runODE(t_eval, x, params, ctrl_params, self.dX_dt))
        self.runODEZ = jit(lambda t_eval, x, params, ctrl_params: runODEZ(t_eval, x, params, ctrl_params, self.dXZ_dt))

        # jit compile matrix operations
        def SSE_next(Y_error, G, Ainv):
            return jnp.sum(Y_error**2) + jnp.einsum('tki,ij,tlj->', G, Ainv, G)
        self.SSE_next = jit(SSE_next)

        def yCOV_next(Y_error, G, Ainv):
            return jnp.einsum('tk,tl->kl', Y_error, Y_error) + jnp.einsum('tki,ij,tlj->kl', G, Ainv, G)
        self.yCOV_next = jit(yCOV_next)

        def A_next(G, Beta):
            return jnp.einsum('tki, kl, tlj->ij', G, Beta, G)
        self.A_next = jit(A_next)

        def GAinvG(G, Ainv):
            return jnp.einsum('tki,ij,tlj->tkl', G, Ainv, G)
        self.GAinvG = jit(GAinvG)

        def NewtonStep(A, g):
            return jnp.linalg.solve(A,g)
        self.NewtonStep = jit(NewtonStep)

        def eval_grad_NLP(Y_error, Beta, G):
            return jnp.einsum('tk,kl,tli->i', Y_error, Beta, G)
        self.eval_grad_NLP = jit(eval_grad_NLP)

    def fit(self, evidence_tol=1e-3, beta_tol=1e-3):
        # estimate parameters using gradient descent
        convergence = np.inf
        previdence  = -np.inf

        while convergence > evidence_tol:
            # update Alpha and Beta hyper-parameters
            self.update_precision()
            # fit using updated Alpha and Beta
            self.res = minimize(fun=self.objective, x0=self.params,
                       jac=True, hess=self.hessian, tol=beta_tol,
                       method='Newton-CG',
                       callback=self.callback)
            if self.verbose:
                print(self.res)
            self.params = self.res.x
            # update covariance
            self.update_covariance()
            # update evidence
            self.update_evidence()
            # check convergence
            convergence = np.abs(previdence - self.evidence) / np.max([1.,np.abs(self.evidence)])
            # update evidence
            previdence = np.copy(self.evidence)

    # EM algorithm to update hyper-parameters
    def update_precision(self):
        print("Updating precision...")

        # initialize precision parameters
        self.N = []
        self.beta = []
        self.Beta = []

        # loop over datasets
        for i,dataset in enumerate(self.datasets):
            # loop over each sample in dataset
            SSE = 0.
            yCOV = 0.
            N = 0
            for t_eval, Y_init, Y_compressed, ctrl_params in dataset:
                
                # count number of observations
                N += len(t_eval[1:]) * np.sum(np.std(Y_compressed, 0) > 0) / self.n_sys_vars

                # run model using current parameters
                if self.A is None:
                    # run ODE on initial condition 
                    output = self.runODE(t_eval, Y_init, self.params, ctrl_params)

                    # Determine SSE
                    Y_error = self.compressors[i](output) - Y_compressed
                    SSE  += np.sum(Y_error**2)
                    yCOV += np.einsum('tk,tl->kl', Y_error, Y_error)
                else:
                    # run model using current parameters, output = [n_time, n_sys_vars]
                    output = self.runODEZ(t_eval, Y_init, self.params, ctrl_params)
                    Y_predicted = output[:, :self.n_sys_vars]

                    # collect gradients and reshape
                    G = np.reshape(output[:, self.n_sys_vars:],
                                  [output.shape[0], self.n_sys_vars, self.n_params])
                    
                    # compress model output and gradient 
                    G = np.einsum('tck,tki->tci', self.compressor_primes[i](Y_predicted), G) 
                    Y_predicted = self.compressors[i](Y_predicted)
                    
                    # Determine SSE
                    Y_error = Y_predicted - Y_compressed
                    SSE  += self.SSE_next(Y_error, G, self.Ainv)
                    yCOV += self.yCOV_next(Y_error, G, self.Ainv)

            ### M step: update hyper-parameters ###
            self.N.append(N)
            if self.A is None:
                # target precision
                self.Beta.append(N*np.linalg.inv(yCOV + 2.*self.b*np.eye(Y_compressed.shape[-1])))
            else:
                # maximize complete data log-likelihood w.r.t. beta
                self.beta.append(N*Y_compressed.shape[-1]/(SSE + 2.*self.b))
                Beta = N*np.linalg.inv(yCOV + 2.*self.b*np.eye(Y_compressed.shape[-1]))
                Beta = (Beta + Beta.T)/2.
                self.Beta.append(Beta)
                # self.Beta.append(N/(SSE + 2.*self.b)*np.eye(Y_compressed.shape[-1]))

        ### M step: update hyper-parameters ###
        if self.A is None:
            # initial guess of parameter precision
            self.alpha = self.alpha_0
            self.Alpha = self.alpha_0*np.ones(self.n_params)
        else:
            # maximize complete data log-likelihood w.r.t. alpha and beta
            self.alpha = self.n_params/(np.dot(self.params-self.prior, self.params-self.prior) + np.trace(self.Ainv) + 2.*self.a)
            #self.Alpha = self.alpha*np.ones(self.n_params)
            self.Alpha = 1./((self.params-self.prior)**2 + np.diag(self.Ainv) + 2.*self.a)

        if self.verbose:
            print("Total samples: {:.0f}, Updated regularization: {:.2e}".format(np.sum(self.N), self.alpha))

    def objective(self, params):
        # compute residuals
        self.RES = 0.
        # compute negative log posterior (NLP)
        self.NLP = np.sum(self.Alpha * (params-self.prior)**2) / 2.
        # compute gradient of negative log posterior
        self.grad_NLP = self.Alpha*(params-self.prior)

        # compute Hessian, covariance of y, sum of squares error
        self.A = np.diag(self.Alpha)

        for i, dataset in enumerate(self.datasets):
            # loop over each sample in dataset
            for t_eval, Y_init, Y_compressed, ctrl_params in dataset:            
                
                # run model using current parameters, output = [n_time, n_sys_vars]
                output = self.runODEZ(t_eval, Y_init, params, ctrl_params)
                Y_predicted = output[:, :self.n_sys_vars]

                # collect gradients and reshape
                G = np.reshape(output[:, self.n_sys_vars:],
                              [output.shape[0], self.n_sys_vars, self.n_params])

                # compress model output and gradient 
                G = np.einsum('tck,tki->tci', self.compressor_primes[i](Y_predicted), G) 
                Y_predicted = self.compressors[i](Y_predicted)
                
                # Determine error
                Y_error = Y_predicted - Y_compressed 

                # compute Hessian
                self.A += self.A_next(G, self.Beta[i])

                # Determine SSE and gradient of SSE
                self.NLP += np.einsum('tk,kl,tl->', Y_error, self.Beta[i], Y_error)/2.
                self.RES += np.sum(Y_error)/self.N[i]

                # sum over time and outputs to get gradient w.r.t params
                self.grad_NLP += self.eval_grad_NLP(Y_error, self.Beta[i], G)

        # make sure precision is symmetric
        self.A = (self.A + self.A.T)/2.

        # return NLP and gradient of NLP
        return self.NLP, self.grad_NLP

    def update_covariance(self):
        # update parameter covariance matrix given current parameter estimate
        if self.A is None:
            self.A = self.alpha_0*np.eye(self.n_params)
        else:
            self.A = np.diag(self.Alpha)

        # loop over datasets
        for i, dataset in enumerate(self.datasets):
            # loop over each sample in dataset
            for t_eval, Y_init, Y_compressed, ctrl_params in dataset:

                # run model using current parameters, output = [n_time, n_sys_vars]
                output = self.runODEZ(t_eval, Y_init, self.params, ctrl_params)
                Y_predicted = output[:, :self.n_sys_vars]

                # collect gradients and reshape
                G = np.reshape(output[:, self.n_sys_vars:],
                              [output.shape[0], self.n_sys_vars, self.n_params])

                # compress model output and gradient 
                G = np.einsum('tck,tki->tci', self.compressor_primes[i](Y_predicted), G) 
                
                # compute Hessian
                self.A += self.A_next(G, self.Beta[i])

        # Laplace approximation of posterior covariance
        self.A = (self.A + self.A.T)/2.
        self.Ainv = np.linalg.inv(self.A)
        self.Ainv = (self.Ainv + self.Ainv.T)/2.

    def update_evidence(self):
        # compute evidence
        self.evidence = np.sum(np.log(self.Alpha))/2. - \
                        np.sum(np.log(np.linalg.eigvalsh(self.A)))/2. - \
                        self.NLP

        # loop over precision matrices from each dataset
        for N, Beta in zip(self.N, self.Beta):
            self.evidence += N*np.sum(np.log(np.linalg.eigvalsh(Beta)))/2.
        print("Evidence {:.3f}".format(self.evidence))

    def jacobian(self, params):
        # compute gradient of cost function
        return self.grad_NLP

    def hessian(self, params):
        # compute hessian of NLP
        return self.A

    def callback(self, xk, res=None):
        if self.verbose:
            print("Total weighted fitting error: {:.3f}".format(self.NLP))
        return True

    def predict(self, x_test, teval, ctrl_params=[], compressor=-1):
        # check if precision has been computed
        if self.A is None:
            self.update_covariance()

        # make predictions given initial conditions and evaluation times
        output = self.runODEZ(teval, x_test, self.params, ctrl_params)

        # reshape gradient
        G = np.reshape(output[:, self.n_sys_vars:],
                       [output.shape[0], self.n_sys_vars, self.n_params])

        # compress model output
        Y_predicted = output[:, :self.n_sys_vars]

        # calculate variance of each output (dimension = [steps, outputs])
        covariance = 1./self.beta[compressor] + self.GAinvG(G, self.Ainv)
        get_diag = vmap(jnp.diag, (0,))
        stdv = np.sqrt(get_diag(covariance))

        return Y_predicted, stdv, covariance

# Import data

In [4]:
t0_data = pd.read_csv("DTL0/REU03_table_t0_20220811.csv")
tf_data = pd.read_csv("DTL0/REU03_table_tf_20220811.csv")

exp_info = ['Treatments', 'Rep', 'Time']
inputs = ['Inulin', 'Starch', 'Pectin', 'ArGal', 'Gum', 'AmAc', 'pH']
species = ['BAabs', 'BPabs', 'BTabs', 'BUabs', 'PCabs', 'PJabs',
       'ACabs', 'CGabs', 'CHabs', 'FPabs', 'ERabs', 'BHabs', 'RIabs',
       'CSabs', 'EHabs']

# data with replicates
reps_data = pd.concat((t0_data[exp_info+inputs+species], tf_data[exp_info+inputs+species]))
rep1_data = reps_data.iloc[reps_data['Rep'].values==1].sort_values(by=['Treatments', 'Time'])
rep2_data = reps_data.iloc[reps_data['Rep'].values==2].sort_values(by=['Treatments', 'Time'])

# average replicates
avg_data = rep1_data.copy().drop(['Rep'], axis=1)
avg_data[species] = (avg_data[species].values + rep2_data[species].values)/2.

# normalize data 
t0_inds = avg_data.Time.values == 0.

# normalize values after initial condition 
max_od = 1. # np.max(avg_data[species].iloc[~t0_inds].values, 0)  
species_inds = np.in1d(avg_data.columns.values, species)
# avg_data.iloc[~t0_inds, species_inds] /= max_od

# set initial conditions 
# avg_data.iloc[t0_inds, species_inds] = np.ceil(avg_data.iloc[t0_inds, species_inds].values) / len(species)

avg_data.describe()

,Time,Inulin,Starch,Pectin,ArGal,Gum,AmAc,pH,BAabs,BPabs,...,PJabs,ACabs,CGabs,CHabs,FPabs,ERabs,BHabs,RIabs,CSabs,EHabs
count,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,...,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000
mean,13.185000,0.195693,0.195882,0.216797,0.195792,0.195836,0.510417,0.510417,0.008104,0.030467,...,0.002928,0.003297,0.002506,0.003631,0.000420,0.000478,0.000548,0.000434,0.003535,0.000346
std,13.254213,0.261085,0.261183,0.283816,0.261136,0.261160,0.497251,0.497251,0.018722,0.051681,...,0.004020,0.006745,0.003926,0.005899,0.000267,0.000492,0.000842,0.000249,0.009137,0.000324
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000667,0.000667,...,0.000667,0.000667,0.000667,0.000667,0.000151,0.000169,0.000116,0.000184,0.000552,0.000020
50%,13.185000,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.750000,0.000667,0.000667,...,0.000667,0.000667,0.000667,0.000667,0.000622,0.000667,0.000667,0.000606,0.000667,0.000398
75%,26.370000,0.333369,0.334456,0.333762,0.333404,0.333616,1.000000,1.000000,0.004173,0.044809,...,0.003485,0.001384,0.002288,0.004315,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667
max,26.370000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.110081,0.241349,...,0.023777,0.044670,0.015978,0.027096,0.000667,0.004437,0.007572,0.000667,0.050191,0.000667


In [5]:
# import mono-culture data
mono_data = pd.read_csv("DTL0/REU03_table_timeSeriesData_20220811.csv")

# data with replicates
reps_data = mono_data[exp_info+inputs+['OD']].copy()
rep1_data = reps_data.iloc[reps_data['Rep'].values==1].sort_values(by=['Treatments', 'Time'])
rep2_data = reps_data.iloc[reps_data['Rep'].values==2].sort_values(by=['Treatments', 'Time'])

# average replicates
avg_mono_data = rep1_data.copy().drop(['Rep'], axis=1)
avg_mono_data.describe()

,Time,Inulin,Starch,Pectin,ArGal,Gum,AmAc,pH,OD
count,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000
mean,13.350218,0.195693,0.195882,0.216797,0.195792,0.195836,0.510417,0.510417,0.138112
std,8.113401,0.259915,0.260012,0.282544,0.259966,0.259990,0.495023,0.495023,0.124472
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.001833
25%,6.375556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038898
50%,13.376944,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.750000,0.098247
75%,20.378611,0.333369,0.334456,0.333762,0.333404,0.333616,1.000000,1.000000,0.204613
max,26.379167,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.547167


In [6]:
avg_mono_data

,Treatments,Time,Inulin,Starch,Pectin,ArGal,Gum,AmAc,pH,OD
0,REU03_1,0.000000,0.500000,0.0,0.000000,0.000000,0.5,0.0,1.0,0.010000
1,REU03_1,2.375000,0.500000,0.0,0.000000,0.000000,0.5,0.0,1.0,0.042733
2,REU03_1,4.375278,0.500000,0.0,0.000000,0.000000,0.5,0.0,1.0,0.047980
3,REU03_1,6.375556,0.500000,0.0,0.000000,0.000000,0.5,0.0,1.0,0.055872
4,REU03_1,8.375833,0.500000,0.0,0.000000,0.000000,0.5,0.0,1.0,0.070998
...,...,...,...,...,...,...,...,...,...,...
121,REU03_9,18.378056,0.334311,0.0,0.332844,0.332844,0.0,0.0,0.0,0.016883
122,REU03_9,20.378611,0.334311,0.0,0.332844,0.332844,0.0,0.0,0.0,0.017573
123,REU03_9,22.378611,0.334311,0.0,0.332844,0.332844,0.0,0.0,0.0,0.018705
124,REU03_9,24.378889,0.334311,0.0,0.332844,0.332844,0.0,0.0,0.0,0.019624


In [7]:
# insert initial OD 
#avg_mono_data[species] = np.ceil(avg_data.iloc[t0_inds, species_inds].values[0]) / len(species)
avg_mono_data[species] = avg_data.iloc[t0_inds, species_inds].values[0]
avg_mono_data.head()

,Treatments,Time,Inulin,Starch,Pectin,ArGal,Gum,AmAc,pH,OD,...,PJabs,ACabs,CGabs,CHabs,FPabs,ERabs,BHabs,RIabs,CSabs,EHabs
0,REU03_1,0.000000,0.5,0.0,0.0,0.0,0.5,0.0,1.0,0.010000,...,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667
1,REU03_1,2.375000,0.5,0.0,0.0,0.0,0.5,0.0,1.0,0.042733,...,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667
2,REU03_1,4.375278,0.5,0.0,0.0,0.0,0.5,0.0,1.0,0.047980,...,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667
3,REU03_1,6.375556,0.5,0.0,0.0,0.0,0.5,0.0,1.0,0.055872,...,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667
4,REU03_1,8.375833,0.5,0.0,0.0,0.0,0.5,0.0,1.0,0.070998,...,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667


In [8]:
# Define function to make predictions on test data

def test_model(model, df_test, max_od, species, plot=False):
    all_treatments = df_test.Treatments.values
    unique_treatments = np.unique(all_treatments)
    numspecies = len(species)

    # save true and predicted values
    y_true = []
    y_pred = []
    y_std  = []
    test_treatments = []
    test_times = []
    all_species_names = []

    # pull a random community trajectory
    for treatment in unique_treatments:
        comm_inds = np.in1d(df_test['Treatments'].values, treatment)
        comm_data = df_test.iloc[comm_inds].copy()

        # make sure comm_data is sorted in chronological order
        comm_data.sort_values(by='Time', ascending=True, inplace=True)
        tspan = comm_data.Time.values

        # pull just the community data
        output_true = comm_data[species].values

        # run model using parameters
        x_test = np.copy(output_true[0, :])
        
        # control parameters 
        ctrl_params = comm_data[inputs].values #[0]

        # test full community
        output, stdv, COV = model.predict(x_test, tspan, ctrl_params=ctrl_params)
        
        # un-normalize
        output_true *= max_od
        output *= max_od
        stdv   *= max_od

        # save predictions after initial value 
        for i, (true, pred, std) in enumerate(zip(output_true[1:], output[1:], stdv[1:])):
            y_true += list(true)
            y_pred += list(pred)
            y_std  += list(std)
            test_times += [tspan[i+1]]*numspecies
            all_species_names += list(species)
            test_treatments += [treatment]*numspecies

        if plot:
            # increase teval
            t_eval = np.linspace(0, tspan[-1]+1)
            steps = len(t_eval)
            output, stdv, COV = model.predict(x_test, t_eval, ctrl_params=ctrl_params)   
            
            # un-normalize
            output *= max_od
            stdv   *= max_od

            # plot the results
            plt.figure(figsize=(9, 6))
            ylim = 0
            for i in range(numspecies):
                out = output[:,i]
                out_true = output_true[:, i]
                std = stdv[:, i]
                if ylim < np.max([np.max(out) + np.max(std)+.1, np.max(out_true)+.1]):
                    ylim = np.max([np.max(out) + np.max(std)+.1, np.max(out_true)+.1])
                if out[0] > 0:
                    plt.scatter(tspan, out_true, color='C{}'.format(i))
                    plt.plot(t_eval, out, label="Predicted species " + str(i+1), color='C{}'.format(i))
                    plt.fill_between(t_eval, out-std, out+std, color='C{}'.format(i), alpha=0.2)

            plt.xlabel("time", fontsize=16)
            plt.ylabel("Abundance", fontsize=16)
            plt.legend(loc='upper left')
            plt.ylim([0, np.min([ylim, 3])])
            plt.title(f"Treatment {treatment} predictions")
            #plt.savefig("Kfold/Figures/{}_{}.pdf".format(dataset.replace("_",""), treatment.replace("<","")))
            #plt.close()
            plt.show()

    return test_treatments, test_times, all_species_names, y_true, y_pred, y_std

In [9]:
# system dimensions
ns = len(species)
nu = len(inputs)
nx = ns + nu

# hidden dimension
nh = 8

# map to hidden dimension
stdv = 1./np.sqrt(nx)
A = np.random.uniform(0, stdv, [nh, nx])

# init bias term
a = np.random.uniform(0, stdv, nh)

# map back to original dimension
stdv = 1./np.sqrt(nh)
B = np.random.uniform(-stdv, 0, [ns, nh])

# init growth rates
b = np.random.uniform(0, stdv, ns)

# init carrying capacities 
t0_inds = avg_data.Time.values == 0.
c = 1./np.max(avg_data[species].values, 0)

# concatenate parameters 
params = np.concatenate((A.flatten(), a, B.flatten(), b, c))
prior  = np.zeros_like(params)
prior[-ns:] = c

n_params = len(params)
n_params

334

In [10]:
# using NODE model 
def system(t, s, params, ctrl_params): 

    # append species to ctrl params
    x = jnp.concatenate((s, ctrl_params[0]))
    
    # map to hidden dimension
    A = jnp.reshape(params[:nh*nx], [nh,nx])
    a = params[nh*nx:nh*nx+nh]

    # map back to original dimension
    B = jnp.reshape(params[nh*nx+nh:nh*nx+nh+nh*ns], [ns,nh])
    b = params[nh*nx+nh+nh*ns:nh*nx+nh+nh*ns+ns]

    # carrying capacity
    c = params[nh*nx+nh+nh*ns+ns:nh*nx+nh+nh*ns+ns+ns]
    
    # compute hidden dimension
    h = jnp.tanh(A@x + a)

    # rate of change of species 
    dsdt = s * (B@h + b) * (1. - c*s)

    return dsdt

In [11]:
# define compression functions 
compressor0 = lambda x: jnp.expand_dims(jnp.sum(x*max_od), 0)     # sum over outputs 
compressor1 = lambda x: x

compressors = [compressor0, compressor1]

## Define ODE (time, x, parameters, u(t), control parameters)

In [12]:
# pull treatment names 
all_mono_treatments = avg_mono_data.Treatments.values
all_treatments = avg_data.Treatments.values
unique_treatments = np.unique(all_treatments)

# set up kfold iterator
kf = KFold(n_splits = K) 

# set up list of measured and predicted values
kfold_species_names = []
kfold_y_true = []
kfold_y_pred = []
kfold_y_stdv = []

# iterate over folds 
for train_index, test_index in kf.split(unique_treatments):
    # train_index, test_index = next(iter(kf.split(unique_treatments)))

    # get index of train and test data
    train_inds_mono = np.in1d(all_mono_treatments, unique_treatments[train_index])
    train_inds = np.in1d(all_treatments, unique_treatments[train_index])
    test_inds  = np.in1d(all_treatments, unique_treatments[test_index])

    # pull out train and test data 
    df_train_mono = avg_mono_data.iloc[train_inds_mono].copy()
    df_train = avg_data.iloc[train_inds].copy()
    df_test  = avg_data.iloc[test_inds].copy()

    # instantiate gLV fit 
    model = ODE(system = system, 
                dataframes=[df_train_mono, df_train],
                compressors = compressors,
                params = params, 
                prior = prior,
                sys_vars = species,
                measured_vars = [['OD'], species],
                controls = inputs,
                verbose=True)
    
    # fit to data 
    t0 = time.time()
    model.fit(evidence_tol=1e-3, beta_tol=1e-4)
    print("Elapsed time {:.2f}s".format(time.time()-t0))

    # predict held-out data
    test_treatments, test_times, all_species_names, y_true, y_pred, y_std = test_model(model, df_test, max_od, species, plot=False)
    kfold_species_names += all_species_names
    kfold_y_true += y_true
    kfold_y_pred += y_pred
    kfold_y_stdv += y_std

[ 0.          2.375       4.37527778  6.37555556  8.37583333 10.37638889
 12.37666667 14.37722222 16.37777778 18.37805556 20.37861111 22.37861111
 24.37888889 26.37916667]
[ 0.   26.37]


NameError: name 'stop' is not defined

In [ ]:
test_treatments, test_times, all_species_names, y_true, y_pred, y_std = test_model(model, avg_data, max_od, species, plot=True)

In [ ]:
r_vals = []
plt.figure(figsize=(9,8))
for s in species:
    y_inds = np.in1d(all_species_names, s)
    y_s_true = np.array(y_true)[y_inds]
    y_s_pred = np.array(y_pred)[y_inds]
    
    r = linregress(y_s_true, y_s_pred).rvalue
    r_vals.append(r)
    plt.scatter(y_s_true, y_s_pred, label=s.replace("abs","")+" R={:.2f}".format(r))
plt.legend()
plt.xlabel("Measured", fontsize=18)
plt.ylabel("Predicted", fontsize=18)

plt.savefig("Results/node_fit_mf.pdf", dpi=200)
plt.show()

In [ ]:
plt.figure(figsize=(9,8))
r_vals = []
for s in species:
    y_inds = np.in1d(kfold_species_names, s)
    y_s_true = np.array(kfold_y_true)[y_inds]
    y_s_pred = np.array(kfold_y_pred)[y_inds]
    
    r = linregress(y_s_true, y_s_pred).rvalue
    r_vals.append(r)
    plt.scatter(y_s_true, y_s_pred, label=s.replace("abs","")+" R={:.2f}".format(r))
plt.legend()
#plt.ylim([0,.5])
plt.show()
print(np.median(r_vals))